# Taking advantage of Colab Pro



## Faster GPUs

With Colab Pro you have priority access to our fastest GPUs. For example, you may get a T4 or P100 GPU at times when most users of standard Colab receive a slower K80 GPU. You can see what GPU you've been assigned at any time by executing the following cell.

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jun  6 17:54:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In order to use a GPU with your notebook, select the Runtime > Change runtime type menu, and then set the hardware accelerator dropdown to GPU.

## More memory

With Colab Pro you have the option to access high-memory VMs when they are available. To set your notebook preference to use a high-memory runtime, select the Runtime > 'Change runtime type' menu, and then select High-RAM in the Runtime shape dropdown.

You can see how much memory you have available at any time by running the following code.


In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Longer runtimes

All Colab runtimes are reset after some period of time (which is faster if the runtime isn't executing code). While Colab Pro subscribers still have limits, these will be roughly twice the limits for non-subscribers.

## Resource limits in Colab Pro

Your resources are not unlimited in Colab Pro. To make the most of Colab Pro, please avoid using resources when you don't need them. For example, only use a GPU or high-RAM runtime when required, and close Colab tabs when finished.


## Send us feedback!

If you have any feedback for us, please let us know. The best way to send feedback is by using the Help > 'Send feedback...' menu. If you encounter usage limits in Colab Pro and would be interested in a product with higher usage limits, do let us know.

If you encounter errors or other issues with billing (payments) for Colab Pro, please email colab-billing@google.com.

In [5]:
import os
import random
import numpy as np
import pandas as pd 
from skimage import io
from skimage import color
from PIL import Image
import PIL.Image
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from dask.array.image import imread
from dask import bag, threaded
from dask.diagnostics import ProgressBar
import cv2
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")


from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from IPython.display import Image


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Input
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image 
from keras.layers.normalization import BatchNormalization
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras import optimizers


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [6]:
import pickle
train_image = []
image_label = []

for i in range(10):
  #path = "/content/drive/My Drive/distdrv/cache/mask_r_224_c_224_c_3_class" + str(i) + ".dat"
  path = "/content/drive/My Drive/distdrv/cache/zoommask_r_224_c_224_c_3_class" + str(i) + ".dat"


  print(f'loading pickle files from class = {i}')
  # get orig image
  file = open(path, 'rb')
  images, labels = pickle.load(file)
  train_image = train_image + images
  #image_label = image_label + labels





loading pickle files from class = 0
loading pickle files from class = 1
loading pickle files from class = 2
loading pickle files from class = 3
loading pickle files from class = 4
loading pickle files from class = 5
loading pickle files from class = 6
loading pickle files from class = 7
loading pickle files from class = 8
loading pickle files from class = 9


In [0]:
images = []
labels = []
driver_details = []

In [8]:
print(f'train image size = {len(train_image)}')

train image size = 22424


In [0]:
import random

random.shuffle(train_image)



In [10]:
print(f'train image size = {len(train_image)}')

train image size = 22424


In [0]:
## getting list of driver names

D = []
for features,labels,drivers in train_image:
    D.append(drivers)

## Deduplicating drivers

deduped = []

for i in D:
    if i not in deduped:
        deduped.append(i)
    

## selecting random drivers for the validation set
driv_selected = []
import random
driv_nums = random.sample(range(len(deduped)), 4)
for i in driv_nums:
    driv_selected.append(deduped[i])


In [12]:
driv_nums

[23, 22, 17, 5]

In [13]:
len(deduped)

26

In [14]:
## Splitting the train and test

X_train= []
y_train = []
X_test = []
y_test = []
D_train = []
D_test = []

#for features,labels,drivers,features2,labels2,drivers2 in zip(train_image, train_image2):
for features, labels, drivers in train_image:

    if drivers in driv_selected:
        X_test.append(features)
        y_test.append(labels)
        #D_test.append(drivers)
    
    else:
        X_train.append(features)
        y_train.append(labels)
        #D_train.append(drivers)


true_test = y_test
    
print (len(X_train),len(X_test))
print (len(y_train),len(y_test))






18899 3525
18899 3525


In [15]:
len(X_train)

18899

In [16]:
## Converting images to nparray. Encoding the Y

X_train = np.array(X_train).reshape(-1,224,224,3)
X_test = np.array(X_test).reshape(-1,224,224,3)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)


(18899, 224, 224, 3)
(3525, 224, 224, 3)
(18899, 10)
(3525, 10)


In [0]:
from __future__ import print_function  # for Python2
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
  #print(var, sys.getsizeof(obj))
  pass

In [18]:
## Defining the input
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
base_model.summary()



17227776/17225924 [==============================] - 0s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________

In [19]:
x=base_model.output
x=GlobalAveragePooling2D()(x)

x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dropout(0.1)(x) # ****reduce dropout 

x=Dense(1024,activation='relu')(x) #dense layer 2

x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x=Dense(512,activation='relu')(x) #dense layer 3

preds=Dense(10,activation='softmax')(x) #final layer with softmax activation

model = Model(inputs=base_model.input, outputs=preds)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128 

In [0]:
from keras import optimizers  

#adam = optimizers.Adam(lr=0.001) #tried 0.0005 - too slow and didn't converge
sgd = optimizers.SGD(lr = 0.005) # try 0.01 - didn't converge and 0.005 , 0.001 best acc of 11%

model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy']) # create object


In [0]:
#model.fit(X_train, y_train, epochs=25, validation_data=(X_test, y_test))

In [22]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping

checkpointer = ModelCheckpoint('/content/drive/My Drive/kaggle/mobilenetmaskModel_aug.h5', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)


datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30
        )
#datagen.fit(X_train)
data_generator = datagen.flow(X_train, y_train, batch_size = 64)

# Fits the model on batches with real-time data augmentation:
mobilenet_model = model.fit_generator(data_generator,steps_per_epoch = len(X_train) / 64, callbacks=[checkpointer, earlystopper],
                                                            epochs = 25, verbose = 1, validation_data = (X_test, y_test))

#vgg16_model = singleModel.fit_generator(data_generator,steps_per_epoch = len(X_train) / 64,
#                                                            epochs = 25, verbose = 1, validation_data = (X_test, y_test))



Epoch 1/25
296/295 [==============================] - 201s 680ms/step - loss: 2.2744 - accuracy: 0.2527 - val_loss: 1.3370 - val_accuracy: 0.5696

Epoch 00001: val_loss improved from inf to 1.33696, saving model to /content/drive/My Drive/kaggle/mobilenetmaskModel_aug.h5
Epoch 2/25
296/295 [==============================] - 187s 631ms/step - loss: 1.3233 - accuracy: 0.5497 - val_loss: 0.8143 - val_accuracy: 0.7262

Epoch 00002: val_loss improved from 1.33696 to 0.81432, saving model to /content/drive/My Drive/kaggle/mobilenetmaskModel_aug.h5
Epoch 3/25
296/295 [==============================] - 189s 639ms/step - loss: 0.9590 - accuracy: 0.6736 - val_loss: 0.6962 - val_accuracy: 0.7770

Epoch 00003: val_loss improved from 0.81432 to 0.69621, saving model to /content/drive/My Drive/kaggle/mobilenetmaskModel_aug.h5
Epoch 4/25
296/295 [==============================] - 192s 650ms/step - loss: 0.7851 - accuracy: 0.7305 - val_loss: 0.5931 - val_accuracy: 0.8017

Epoch 00004: val_loss improve

In [0]:
#vgg16_model.save("/content/drive/My Drive/kaggle/singleModel_aug.h5")
model.save_weights("/content/drive/My Drive/kaggle/mobilenetmaskModel_aug_weights.h5")

In [30]:
from keras.models import load_model
rcModel = load_model('/content/drive/My Drive/kaggle/mobilenetmaskModel_aug.h5')
rcModel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128 

In [31]:
X_test.shape

(3525, 224, 224, 3)

In [32]:
# labels is the image array

from sklearn.metrics import accuracy_score, confusion_matrix

model1_prediction = []
model1_pred_class = []

model1_prediction = rcModel.predict(X_test)
print('Images Predicted until now:',len(model1_prediction))
print(f'True images: {len(true_test)}')

for i in range(len(model1_prediction)):
    model1_pred_class.append(np.where(model1_prediction[i] == np.amax(model1_prediction[i]))[0][0])

    
print('The accuracy of this model over validation set is:',accuracy_score(true_test,model1_pred_class))
confusion_matrix(true_test,model1_pred_class)

Images Predicted until now: 3525
True images: 3525
The accuracy of this model over validation set is: 0.8451063829787234


array([[258,   0,   1,   0,   0,  73,   9,   0,  48,  12],
       [  9, 270,   0,   2,   0,   0,  19,   3,   5,   6],
       [  0,   0, 354,   0,   0,   0,   2,   0,   3,   0],
       [ 14,   1,   0, 352,   5,   0,   0,   1,  15,   9],
       [  0,   0,   0,   1, 322,   0,   0,   1,  60,   0],
       [  0,   1,   0,   0,   0, 373,   1,  11,   9,   3],
       [  5,   0,   0,   0,   0,   0, 353,   0,  36,   2],
       [  0,   0,   1,   0,   0,   0,   0, 302,  13,   3],
       [ 19,   0,   0,   0,   5,   3,  26,   0, 176,   9],
       [ 14,   0,   1,   0,   0,  22,   4,   5,  54, 219]])

In [33]:
print(f'prediction={model1_prediction}, shape={model1_prediction.shape}')

prediction=[[2.0169340e-05 4.2288161e-06 4.1107305e-06 ... 2.0928520e-01
  7.8923178e-01 1.0349319e-03]
 [1.7624736e-01 2.7597108e-04 1.1436696e-07 ... 4.8707981e-05
  1.0139478e-03 8.2195240e-01]
 [3.7138455e-04 1.9715612e-06 3.1026437e-05 ... 2.2980712e-05
  2.6279952e-02 5.7266923e-05]
 ...
 [2.2652991e-08 2.3407394e-09 3.6930967e-07 ... 1.3056524e-07
  6.6985311e-03 3.9002191e-07]
 [2.1893689e-07 1.1701079e-09 4.6203218e-07 ... 1.0724912e-08
  9.2967518e-04 1.8225170e-06]
 [2.9705213e-03 2.3489829e-06 1.3949519e-05 ... 6.8215840e-03
  1.0880026e-02 4.2447406e-03]], shape=(3525, 10)


In [34]:
from sklearn.metrics import log_loss

lgloss = log_loss(y_test, model1_prediction, eps=1e-15, normalize=True, sample_weight=None, labels=None)

print('The log loss from this model is:',round(lgloss,2))

The log loss from this model is: 0.48
